In [2]:

import requests
import json
import time
import pandas as pd

## Exploration and Testing

In [ ]:
itemid = '4755'
item = requests.get(f'https://services.runescape.com/m=itemdb_rs/api/catalogue/detail.json?item={itemid}')

In [ ]:
item

<Response [200]>

In [ ]:
item.json()

{'item': {'icon': 'https://secure.runescape.com/m=itemdb_rs/1730114200891_obj_sprite.gif?id=4755',
  'icon_large': 'https://secure.runescape.com/m=itemdb_rs/1730114200891_obj_big.gif?id=4755',
  'id': 4755,
  'type': 'Melee weapons - high level',
  'typeIcon': 'https://www.runescape.com/img/categories/Melee weapons - high level',
  'name': "Verac's flail",
  'description': "Verac the Defiled's flail.",
  'current': {'trend': 'neutral', 'price': '1.1m'},
  'today': {'trend': 'neutral', 'price': 0},
  'members': 'true',
  'day30': {'trend': 'negative', 'change': '-0.0%'},
  'day90': {'trend': 'positive', 'change': '+0.0%'},
  'day180': {'trend': 'negative', 'change': '-6.0%'}}}

In [ ]:
parsed = item.json()
parsed.keys()

dict_keys(['item'])

In [ ]:
dir(parsed)

['__class__',
 '__class_getitem__',
 '__contains__',
 '__delattr__',
 '__delitem__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__ior__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__ne__',
 '__new__',
 '__or__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__reversed__',
 '__ror__',
 '__setattr__',
 '__setitem__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'clear',
 'copy',
 'fromkeys',
 'get',
 'items',
 'keys',
 'pop',
 'popitem',
 'setdefault',
 'update',
 'values']

In [ ]:

type(parsed['item']['current']['price'])

str

In [3]:
itemids = ['10284','10156','8880','4755','4587','4151','19323','21777','42767','1038','1042','1048']
itemname = []
itemtype = []
itemprice = []
item30daytrend = []
item90daytrend = []
item180daytrend = []
for i in itemids:
    item = requests.get(f'https://services.runescape.com/m=itemdb_rs/api/catalogue/detail.json?item={i}')
    parse = item.json()
    itemname.append(parse['item']['name'])
    itemtype.append(parse['item']['type'])
    if 'm' in parse['item']['current']['price']:
      num = parse['item']['current']['price']
      split = num.split('m')
      split = float(split[0])
      split = split * 10**6
      itemprice.append(split)
    elif 'b' in parse['item']['current']['price']:
      num = parse['item']['current']['price']
      split = num.split('b')
      split = float(split[0])
      split = split * 10**9
      itemprice.append(split)
    elif 'k' in parse['item']['current']['price']:
      num = parse['item']['current']['price']
      split = num.split('k')
      split = float(split[0])
      split = split * 10**3
      itemprice.append(split)
    else:
      itemprice.append(parse['item']['current']['price'])
    item30daytrend.append(parse['item']['day30']['change'])
    item90daytrend.append(parse['item']['day90']['change'])
    item180daytrend.append(parse['item']['day180']['change'])
    time.sleep(5)


In [37]:
df = pd.DataFrame({
    'name': itemname,
    'type': itemtype,
    'price': itemprice,
    '30daytrend': item30daytrend,
    '90daytrend': item90daytrend,
    '180daytrend': item180daytrend
})


In [38]:
df.head()

,name,type,price,30daytrend,90daytrend,180daytrend
0,Magic composite bow,Ranged weapons,1200000.0,+1.0%,+1.0%,-11.0%
1,Hunters' crossbow,Ranged weapons,"9,885",0.0%,+2.0%,+2.0%
2,Dorgeshuun crossbow,Ranged weapons,"4,621",0.0%,0.0%,+1.0%
3,Verac's flail,Melee weapons - high level,1100000.0,+0.0%,+0.0%,-3.0%
4,Dragon scimitar,Melee weapons - mid level,98200.0,+1.0%,+1.0%,-0.0%


In [40]:
new = df.convert_dtypes(convert_floating=True)
new.loc[[1, 2], 'price'] = new.loc[[1, 2], 'price'].str.replace(',', '').astype(int)
new.dtypes

,0
name,string[python]
type,string[python]
price,object
30daytrend,string[python]
90daytrend,string[python]
180daytrend,string[python]


In [41]:
new['30daytrend'] = new['30daytrend'].str.replace('%','')
new['90daytrend'] = new['90daytrend'].str.replace('%','')
new['180daytrend'] = new['180daytrend'].str.replace('%','')
new['30daytrend'] = pd.to_numeric(new['30daytrend'], errors='coerce')
new['90daytrend'] = pd.to_numeric(new['90daytrend'], errors='coerce')
new['180daytrend'] = pd.to_numeric(new['180daytrend'], errors='coerce')

In [42]:
new

,name,type,price,30daytrend,90daytrend,180daytrend
0,Magic composite bow,Ranged weapons,1200000.0,1.0,1.0,-11.0
1,Hunters' crossbow,Ranged weapons,9885,0.0,2.0,2.0
2,Dorgeshuun crossbow,Ranged weapons,4621,0.0,0.0,1.0
3,Verac's flail,Melee weapons - high level,1100000.0,0.0,0.0,-3.0
4,Dragon scimitar,Melee weapons - mid level,98200.0,1.0,1.0,-0.0
5,Abyssal whip,Melee weapons - high level,84600.0,-18.0,-11.0,-1.0
6,Dragon staff,Magic weapons,1300000.0,0.0,-1.0,-4.0
7,Armadyl battlestaff,Miscellaneous,136600000.0,-2.0,-1.0,11.0
8,Merethiel's stave,Miscellaneous,3900000.0,0.0,-4.0,-9.0
9,Red partyhat,Costumes,53700000000.0,0.0,4.0,15.0


## Constructing the Update Function

In [43]:
new.to_csv('RSitem_data.csv', index=False)